In [ ]:
from RawHandler.RawHandlerRawpy import RawHandlerRawpy
from RawHandler.utils import download_file_requests
import matplotlib.pyplot as plt
import os

In [ ]:
# Download example raw files
def download_file_if_needed(file_url, output_file):
    if not os.path.exists(output_file):
        download_file_requests(file_url, output_file)


file_url = "https://dataverse.uclouvain.be/api/access/datafile/:persistentId?persistentId=doi:10.14428/DVN/DEQCIM/N9OG4V"
output_file = "wall.raf"
download_file_if_needed(file_url, output_file)

In [ ]:
# Load the first image
rh = RawHandlerRawpy("wall.raf", colorspace="sRGB")

In [ ]:
# This wraps rawpy's postprocess method and gives an XYZ colorspace image
dims = (1800, 1800 + 200, 2700, 2700 + 200)
linear = rh._input_handler(dims=dims)
plt.imshow(linear.transpose(1, 2, 0) ** (1 / 2.2))

In [ ]:
# This gives a color space transformed image
dims = (1800, 1800 + 200, 2700, 2700 + 200)
linear = rh.apply_colorspace_transform(dims=dims, clip=True)
plt.imshow(linear.transpose(1, 2, 0) ** (1 / 2.2))

In [ ]:
# We can also simulate a sparse representation
dims = (1800, 1800 + 32, 2700, 2700 + 32)

rgb = rh.as_sparse(dims=dims, pattern="RGGB", cfa_type="bayer")
plt.imshow((rgb).transpose(1, 2, 0) ** (1 / 2.2))
rgb.min(), rgb.max()

In [ ]:
# Or a simulate a CFA
rgb = rh.as_cfa(dims=dims, pattern="RGGB", cfa_type="bayer")
plt.imshow((rgb).transpose(1, 2, 0) ** (1 / 2.2))
rgb.min(), rgb.max()

In [ ]:
# We can also simulate a sparse representation of an x-trans CFA
dims = (1800, 1800 + 32, 2700, 2700 + 32)

rgb = rh.as_sparse(dims=dims, cfa_type="xtrans", clip=True)
plt.imshow((rgb).transpose(1, 2, 0) ** (1 / 2.2))
rgb.min(), rgb.max()

In [ ]:
# Or a simulate an x-trans CFA
rgb = rh.as_cfa(dims=dims, cfa_type="xtrans", clip=True)
plt.imshow((rgb).transpose(1, 2, 0) ** (1 / 2.2))
rgb.min(), rgb.max()

In [ ]:
# We can also set it so that it always crops a multiple of the shape of the CFA array
dims = (1801, 1800 + 32, 2701, 2700 + 32)

rgb = rh.as_sparse(dims=dims, cfa_type="xtrans", clip=True, safe_crop=6)
plt.imshow((rgb).transpose(1, 2, 0) ** (1 / 2.2))
rgb.min(), rgb.max(), rgb.shape